# Regression Models for each Property Type

IN the last notebook our analysis concluded that the regression errors have different behavior for each property type. This has leaded us to the necessity to handle the regression problem differently for each property type.

In this notebook we'll address this problem by testing different regression models and selecting the best ones for each property type.

In [1]:
import os
import os.path as P
import pickle
import typing as T

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, Lasso, Ridge
from sklearn.metrics import PredictionErrorDisplay, make_scorer, mean_squared_log_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [2]:
sklearn.set_config(transform_output="pandas")

In [3]:
random_state = 42